In [3]:
# imports
import os
import torch
import seaborn as sns
import matplotlib.pyplot as plt

from torch_geometric.data import Data
from utils import ROOT_DIR, NEURON_LABELS
from visualize._utils import draw_connectome
from torch_geometric.utils import to_dense_adj

from tqdm import tqdm
import pandas as pd

import numpy as np

In [4]:
files = ["graph_tensors_cook2019.pt", "graph_tensors_openworm.pt", "graph_tensors_white1986_jse.pt", 
         "graph_tensors_white1986_jsh.pt", "graph_tensors_white1986_n2u.pt", "graph_tensors_white1986_whole.pt",
         "graph_tensors_witvliet2020_7.pt", "graph_tensors_witvliet2020_8.pt"]

all_data = []

master_sheet = pd.read_csv('../data/raw/neuron_master_sheet.csv')

for file in files:
    graph_tensors = torch.load(
        os.path.join(ROOT_DIR, "data", "processed", "connectome", file)
    )  # options: "graph_tensors.pt", "graph_tensors_witvliet2020_7.pt", "graph_tensors_witvliet2020_8.pt", "graph_tensors_cook2019.pt", "graph_tensors_openworm.pt",
    # "graph_tensors_white1986_n2u.pt", "graph_tensors_white1986_jsh.pt", "graph_tensors_white1986_jse.pt", "graph_tensors_white1986_whole.pt"

    # make the graph
    graph = Data(**graph_tensors)
    
    edges = graph["edge_index"].T
    edge_attr = graph["edge_attr"]
    node_labels = graph["node_label"]
    node_type = graph["node_type"]
    
    for edge, attr in tqdm(zip(edges, edge_attr)):
        source_connectome = "_".join(file.split("_")[2:])[:-3]
        
        target_neuron_type = node_type[int(graph["y"][edge[1]])]
        origin_neuron = node_labels[int(edge[0])]
        target_neuron = node_labels[int(edge[1])]
        gap_junction_weight = float(attr[0])
        chemical_synapse_weight = float(attr[1])
        
        origin_neuron_type = node_type[int(graph["y"][edge[0]])]
        origin_neuron_class = origin_neuron[:-1] # TODO: NOT HOW YOU GET THE CLASS!
        origin_neuron_neurotransmitter = master_sheet.loc[master_sheet['label'] == origin_neuron].iloc(0)[0]['neurotransmitter']
        origin_neuron_x = graph["pos"][int(edge[0])][0]
        origin_neuron_y = graph["pos"][int(edge[0])][1]
        origin_neuron_z = graph["pos"][int(edge[0])][2]
        
        target_neuron_type = node_type[int(graph["y"][edge[1]])]
        target_neuron_class = target_neuron[:-1] # TODO: NOT HOW YOU GET THE CLASS!
        target_neuron_neurotransmitter = master_sheet.loc[master_sheet['label'] == target_neuron].iloc(0)[0]['neurotransmitter']
        target_neuron_x = graph["pos"][int(edge[1])][0]
        target_neuron_y = graph["pos"][int(edge[1])][1]
        target_neuron_z = graph["pos"][int(edge[1])][2]
        
        
        all_data += [[source_connectome, origin_neuron, target_neuron, gap_junction_weight,
                      chemical_synapse_weight, origin_neuron_type, origin_neuron_class, origin_neuron_neurotransmitter,
                      origin_neuron_x, origin_neuron_y, origin_neuron_z, target_neuron_type, target_neuron_class, target_neuron_neurotransmitter,
                      target_neuron_x, target_neuron_y, target_neuron_z]]
    
    
all_data

46837it [00:35, 1335.54it/s]
47683it [00:32, 1481.88it/s]
44901it [00:31, 1436.33it/s]
45759it [00:31, 1470.45it/s]
45808it [00:31, 1459.83it/s]
46603it [00:31, 1479.58it/s]
46057it [00:31, 1471.07it/s]
46107it [00:31, 1474.35it/s]


[['cook2019',
  'I2L',
  'I1L',
  2.0,
  2.0,
  'pharyngeal',
  'I2',
  'Glu',
  4.85,
  13.08,
  7.23,
  'pharyngeal',
  'I1',
  'ACh',
  7.43,
  0.71,
  4.12],
 ['cook2019',
  'I3',
  'I1L',
  0.0,
  1.0,
  'pharyngeal',
  'I',
  'ACh',
  13.57,
  18.92,
  -0.96,
  'pharyngeal',
  'I1',
  'ACh',
  7.43,
  0.71,
  4.12],
 ['cook2019',
  'I5',
  'I1L',
  0.0,
  2.0,
  'pharyngeal',
  'I',
  'Glu',
  -2.79,
  72.18,
  -2.56,
  'pharyngeal',
  'I1',
  'ACh',
  7.43,
  0.71,
  4.12],
 ['cook2019',
  'MI',
  'I1L',
  0.0,
  1.0,
  'pharyngeal',
  'M',
  'Glu',
  14.61,
  16.72,
  -1.03,
  'pharyngeal',
  'I1',
  'ACh',
  7.43,
  0.71,
  4.12],
 ['cook2019',
  'I2R',
  'I1R',
  1.0,
  1.0,
  'pharyngeal',
  'I2',
  'Glu',
  3.39,
  13.82,
  -6.0,
  'pharyngeal',
  'I1',
  'ACh',
  6.0,
  0.0,
  -2.71],
 ['cook2019',
  'I3',
  'I1R',
  1.0,
  1.0,
  'pharyngeal',
  'I',
  'ACh',
  13.57,
  18.92,
  -0.96,
  'pharyngeal',
  'I1',
  'ACh',
  6.0,
  0.0,
  -2.71],
 ['cook2019',
  'I6',
  'I1R',

In [6]:
keys = ["source_connectome", "origin_neuron", "target_neuron", "gap_junction_weight", "chemical_synapse_weight",
        "origin_neuron_type", "origin_neuron_class", "origin_neuron_neurotransmitter", "origin_neuron_x",
        "origin_neuron_y", "origin_neuron_z",  "target_neuron_type", "target_neuron_class",
        "target_neuron_neurotransmitter", "target_neuron_x", "target_neuron_y", "target_neuron_z"]

dataframe = pd.DataFrame(all_data)
dataframe = dataframe.set_axis(keys, axis=1)
dataframe

,source_connectome,origin_neuron,target_neuron,gap_junction_weight,chemical_synapse_weight,origin_neuron_type,origin_neuron_class,origin_neuron_neurotransmitter,origin_neuron_x,origin_neuron_y,origin_neuron_z,target_neuron_type,target_neuron_class,target_neuron_neurotransmitter,target_neuron_x,target_neuron_y,target_neuron_z
0,cook2019,I2L,I1L,2.0,2.0,pharyngeal,I2,Glu,4.85,13.08,7.23,pharyngeal,I1,ACh,7.43,0.71,4.12
1,cook2019,I3,I1L,0.0,1.0,pharyngeal,I,ACh,13.57,18.92,-0.96,pharyngeal,I1,ACh,7.43,0.71,4.12
2,cook2019,I5,I1L,0.0,2.0,pharyngeal,I,Glu,-2.79,72.18,-2.56,pharyngeal,I1,ACh,7.43,0.71,4.12
3,cook2019,MI,I1L,0.0,1.0,pharyngeal,M,Glu,14.61,16.72,-1.03,pharyngeal,I1,ACh,7.43,0.71,4.12
4,cook2019,I2R,I1R,1.0,1.0,pharyngeal,I2,Glu,3.39,13.82,-6.00,pharyngeal,I1,ACh,6.00,0.00,-2.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369750,witvliet2020_8,VD6,VD8,0.0,0.0,motor,VD,GABA,-15.85,383.75,0.96,motor,VD,GABA,-13.22,482.54,2.60
369751,witvliet2020_8,VD6,VD9,0.0,0.0,motor,VD,GABA,-15.85,383.75,0.96,motor,VD,GABA,-13.48,548.63,-0.01
369752,witvliet2020_8,VD7,VD8,0.0,0.0,motor,VD,GABA,-14.25,431.61,-1.05,motor,VD,GABA,-13.22,482.54,2.60
369753,witvliet2020_8,VD7,VD9,0.0,0.0,motor,VD,GABA,-14.25,431.61,-1.05,motor,VD,GABA,-13.48,548.63,-0.01
